### Домашнее задание по обработке текстов
Предсказание цены акции по экономическим новостям

Входные данные:

    1.Новости о компании "Газпром", начиная с 2010 года
- преообразуем колонку дата в формат даты.    


In [1]:
#!pip install pymystem3
#!pip install rutermextract

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('texts.csv')
df.date =  pd.to_datetime(df.date, format='%d.%m.%Y')
df['len'] = df.text.apply(len)
df.set_index('date', inplace=True)
df.head()

,text,len
date,,
2017-11-09,Компания рассчитывает на решение по газовому с...,419
2017-11-08,"Как и предполагал “Ъ”, «Газпром», воспользова...",624
2017-11-01,Новая редакция американских санкций ставит по...,835
2017-10-30,"Как стало известно “Ъ”, известный на рынке ри...",436
2017-10-23,"НОВАТЭК, который через пять лет собирается за...",1272


    2.Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
        цена открытия (Open)
        цена закрытия (ClosingPrice)
        максимальная цена за день (DailyHigh)
        минимальная цена за день (DailyLow)
        объем бумаг (VolumePcs)
        
- добавим колонку "Дельта" - разница между открытием и закрытием, преобразуем колонки в нужные форматы "Дата" или "Число"

In [60]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all['date'] =  pd.to_datetime(pr_all.Date, format='%d.%m.%Y')
pr_all.drop(['Date'], axis=1, inplace=True)
pr_all['Open'] = pr_all['Open'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['ClosingPrice'] = pr_all['ClosingPrice'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['DailyHigh'] = pr_all['DailyHigh'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['DailyLow'] = pr_all['DailyLow'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['delta'] = pr_all.Open - pr_all.ClosingPrice
pr_all['ClosingPrice_bin'] = pr_all.apply(lambda row: 1 if row.delta >= 0 else 0, axis=1)
pr_all.set_index('date', inplace=True)
pr_all.head(5)

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,ClosingPrice_bin
date,,,,,,,
2017-12-08,133.429993,132.600006,133.899994,132.000000,16037970,0.829987,1
2017-12-07,133.699997,133.020004,133.869995,132.809998,18198430,0.679993,1
2017-12-06,133.330002,134.000000,134.289993,132.910004,14641730,-0.669998,0
2017-12-05,133.479996,133.649994,133.990005,132.779999,12684800,-0.169998,0
2017-12-04,133.009995,133.770004,134.000000,131.929993,17818980,-0.760010,0


#### Часть 1. Вводная

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 

Собираем общую таблицу:

In [61]:
Data = pr_all.join(df)
Data.dropna(axis=0, inplace=True)
Data.head()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,ClosingPrice_bin,text,len
date,,,,,,,,,
2017-11-09,132.309998,131.500000,132.820007,131.139999,33869650,0.809998,1,Компания рассчитывает на решение по газовому с...,419.0
2017-11-08,132.000000,132.300003,133.940002,131.580002,39381960,-0.300003,0,"Как и предполагал “Ъ”, «Газпром», воспользова...",624.0
2017-11-01,126.400002,126.500000,126.889999,125.970001,18232550,-0.099998,0,Новая редакция американских санкций ставит по...,835.0
2017-10-30,125.959999,125.980003,126.930000,125.529999,19263340,-0.020004,0,"Как стало известно “Ъ”, известный на рынке ри...",436.0
2017-10-23,127.050003,126.800003,127.470001,126.370003,17308800,0.250000,1,"НОВАТЭК, который через пять лет собирается за...",1272.0


In [62]:
Data.describe()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,ClosingPrice_bin,len
count,1145.000000,1145.000000,1145.000000,1145.000000,1.145000e+03,1145.000000,1145.000000,1145.000000
mean,150.018681,149.902970,151.635738,148.193511,4.492943e+07,0.115712,0.526638,772.908297
std,23.464697,23.390248,23.702841,23.090757,2.455151e+07,2.316764,0.499508,546.978587
min,107.230003,107.169998,109.199997,106.500000,7.642310e+06,-12.639999,0.000000,85.000000
25%,135.889999,135.770004,137.000000,134.500000,2.859430e+07,-1.120003,0.000000,406.000000
50%,144.270004,144.300003,145.889999,142.669998,3.843521e+07,0.110001,1.000000,584.000000
75%,157.169998,156.690002,158.630005,155.149994,5.580905e+07,1.310013,1.000000,1010.000000
max,243.520004,244.000000,247.470001,241.770004,2.024663e+08,13.849998,1.000000,4120.000000


In [63]:
%%time 
from pymystem3 import Mystem

def lemma(text):
    text = text.replace('\xa0', ' ')
    try:
        return "".join(Mystem().lemmatize(text)).strip()  
    except:
        return " "

Data['lem_text'] = Data.text.apply(lemma)

CPU times: user 1.77 s, sys: 6.66 s, total: 8.43 s
Wall time: 18min 23s


In [64]:
Data.head()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,ClosingPrice_bin,text,len,lem_text
date,,,,,,,,,,
2017-11-09,132.309998,131.500000,132.820007,131.139999,33869650,0.809998,1,Компания рассчитывает на решение по газовому с...,419.0,компания рассчитывать на решение по газовый сп...
2017-11-08,132.000000,132.300003,133.940002,131.580002,39381960,-0.300003,0,"Как и предполагал “Ъ”, «Газпром», воспользова...",624.0,"как и предполагать “Ъ”, «газпром», воспользова..."
2017-11-01,126.400002,126.500000,126.889999,125.970001,18232550,-0.099998,0,Новая редакция американских санкций ставит по...,835.0,новый редакция американский санкция ставить по...
2017-10-30,125.959999,125.980003,126.930000,125.529999,19263340,-0.020004,0,"Как стало известно “Ъ”, известный на рынке ри...",436.0,"как становиться известно “Ъ”, известный на рын..."
2017-10-23,127.050003,126.800003,127.470001,126.370003,17308800,0.250000,1,"НОВАТЭК, который через пять лет собирается за...",1272.0,"новатэк, который через пять год собираться зап..."


In [65]:
D = Data.copy()

In [66]:
D['miller_count'] = D.lem_text.map(
    lambda text: text.count('миллер') if text else 0,
    na_action='ignore'
)
D['sever_count'] = D.lem_text.map(
    lambda text: text.count('северный поток') if text else 0,
    na_action='ignore'
)
D['turk_count'] = D.lem_text.map(
    lambda text: text.count('турецкий поток') if text else 0,
    na_action='ignore'
)


In [67]:
Turk = D[D.turk_count > 0 ].copy()
Turk_count = len(Turk)
Sever = D[D.sever_count > 0 ].copy()
Sever_count = len(Sever)


##### Ответьте на следующие вопросы:

    1.Есть ли корреляция между средней длинной текста за день и ценой закрытия?
    2.Есть ли корреляция между количеством упоминаний Алексея Миллера и ценой закрытия? Учтите разные варианты написания имени.
    3.Упоминаний какого газопровода в статьях больше:
        "северный поток"
        "турецкий поток"?    

In [68]:
display("Количество статей, упоминающих Турецкий поток - " + str(Turk_count))
display("Количество статей, упоминающих Северный поток - " + str(Sever_count))
CorrKoef = D.corr()
CorrKoef

'Количество статей, упоминающих Турецкий поток - 34'

'Количество статей, упоминающих Северный поток - 12'

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,ClosingPrice_bin,len,miller_count,sever_count,turk_count
Open,1.000000,0.995115,0.998069,0.997341,0.371362,0.081451,0.047017,0.014575,0.006844,0.037291,-0.062884
ClosingPrice,0.995115,1.000000,0.997728,0.997670,0.376791,-0.017338,-0.024772,0.012376,0.008110,0.035715,-0.061107
DailyHigh,0.998069,0.997728,1.000000,0.996902,0.396443,0.035520,0.012675,0.015082,0.007792,0.036612,-0.061706
DailyLow,0.997341,0.997670,0.996902,1.000000,0.346697,0.028728,0.011242,0.010711,0.008442,0.037763,-0.059570
VolumePcs,0.371362,0.376791,0.396443,0.346697,1.000000,-0.042877,-0.058108,0.076835,0.007320,-0.011082,-0.100423
delta,0.081451,-0.017338,0.035520,0.028728,-0.042877,1.000000,0.726306,0.022667,-0.012564,0.017115,-0.019967
ClosingPrice_bin,0.047017,-0.024772,0.012675,0.011242,-0.058108,0.726306,1.000000,-0.012253,-0.024272,0.026035,-0.030337
len,0.014575,0.012376,0.015082,0.010711,0.076835,0.022667,-0.012253,1.000000,0.148300,0.081808,0.058477
miller_count,0.006844,0.008110,0.007792,0.008442,0.007320,-0.012564,-0.024272,0.148300,1.000000,0.164734,-0.016549
sever_count,0.037291,0.035715,0.036612,0.037763,-0.011082,0.017115,0.026035,0.081808,0.164734,1.000000,0.075493


1. Судя по коэффициенту корреляции длинна текста не коррелирует ни с ценой закрытия, ни, даже с дельтой за день.
2. Судя по коэффициенту корреляции количество упоминаний Алексея Миллера не коррелирует с ценой закрытия и дельтой за день.
3. Количество статей, упоминающих турецкий поток почти в три раза больше, чем, статей, упоминающих Северный поток.

    4.О каких санкциях пишут в статьях?

In [69]:
! cat texts.csv | docker run --rm -i inemo/syntaxnet_rus > dat.conll

2018-02-25 20:58:14.406273: W external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-25 20:58:14.406311: W external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-25 20:58:14.406325: W external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-25 20:58:14.426369: I syntaxnet/term_frequency_map.cc:101] Loaded 34 terms from ./syntaxnet/models/Russian-SynTagRus/label-map.
2018-02-25 20:58:14.426715: I syntaxnet/embedding_feature_extractor.cc:35] Features: stack.child(1).label stack.child(1).si

2018-02-25 20:58:15.078931: I syntaxnet/term_frequency_map.cc:101] Loaded 18749 terms from ./syntaxnet/models/Russian-SynTagRus/char-ngram-map.
2018-02-25 20:58:15.079016: I syntaxnet/term_frequency_map.cc:101] Loaded 18749 terms from ./syntaxnet/models/Russian-SynTagRus/char-ngram-map.
2018-02-25 20:58:15.091811: I syntaxnet/term_frequency_map.cc:101] Loaded 31 terms from ./syntaxnet/models/Russian-SynTagRus/tag-map.
2018-02-25 20:58:16.618006: I syntaxnet/term_frequency_map.cc:101] Loaded 103473 terms from ./syntaxnet/models/Russian-SynTagRus/word-map.
INFO:tensorflow:Building training network with parameters: feature_sizes: [12 20 20 20] domain_sizes: [    37     66     33 103475]
INFO:tensorflow:Restoring parameters from ./syntaxnet/models/Russian-SynTagRus/parser-params
2018-02-25 20:58:18.185202: I syntaxnet/term_frequency_map.cc:101] Loaded 34 terms from ./syntaxnet/models/Russian-SynTagRus/label-map.
2018-02-25 20:58:18.185259: I syntaxnet/embedding_feature_extractor.cc:35] Fea

2018-02-25 20:58:21.476814: I syntaxnet/term_frequency_map.cc:101] Loaded 18749 terms from ./syntaxnet/models/Russian-SynTagRus/char-ngram-map.
2018-02-25 20:58:21.489717: I syntaxnet/term_frequency_map.cc:101] Loaded 18749 terms from ./syntaxnet/models/Russian-SynTagRus/char-ngram-map.
2018-02-25 20:58:23.597867: I syntaxnet/term_frequency_map.cc:101] Loaded 103473 terms from ./syntaxnet/models/Russian-SynTagRus/word-map.
2018-02-25 20:58:23.666168: I syntaxnet/term_frequency_map.cc:101] Loaded 103473 terms from ./syntaxnet/models/Russian-SynTagRus/word-map.
INFO:tensorflow:Processed 1024 documents
INFO:tensorflow:Processed 1024 documents
INFO:tensorflow:Processed 1024 documents
INFO:tensorflow:Processed 346 documents
INFO:tensorflow:Processed 1024 documents
INFO:tensorflow:Processed 1024 documents
INFO:tensorflow:Total processed documents: 2394
INFO:tensorflow:num correct tokens: 0
INFO:tensorflow:total tokens: 123340
INFO:tensorflow:Seconds elapsed in evaluation: 246.71, eval metric

In [70]:
train_file = 'texts.conll'

In [71]:
%%time
import codecs
import re
processed_sentences = []
sentence = []
for line in codecs.open(train_file, 'r', 'utf-8'):
    if len(line) == 1:
        processed_sentences.append(sentence)
        sentence = []
    else:
        word = line.split("\t")
        sentence.append(word)
deps = []
res = []
sanctions = []
sanctions1 = []
for sentence in processed_sentences:
    s = u""
    for line in sentence:
        s += u"\t".join(line) + u'\n'
    deps.append(s)

for sent_dep in deps:    
    s_deps = sent_dep.split('\n')
    for one_step in s_deps:
        if len(one_step) > 1:
            spl_one_step = one_step.split('\t')
            if 'санкц' in spl_one_step[1] and not('санкционирован' in spl_one_step[1]):
                Word_sanctions = spl_one_step[1]
                reg = re.compile('[^а-яА-Я ]')
                Word_sanctions = reg.sub('', Word_sanctions)                
                number = int(spl_one_step[0]) 
                for two_step in s_deps:
                    spl_two_step = two_step.split('\t')
                    if len(spl_two_step) > 1 :
                        word_type = spl_two_step[3]
                        if word_type == 'ADJ' :
                            word = spl_two_step[1]
                            reg = re.compile('[^а-яА-Я ]')
                            word = reg.sub('', word)
                            words = word #+ ' ' + Word_sanctions
                            if not(word in sanctions1) and len(word)>4:
                                sanctions1.append(word)
                                sanctions.append(words)
print('Всего ' + str(len(sanctions)) + ' упоминаний о санкциях')  
sanctions.sort()

Всего 416 упоминаний о санкциях
CPU times: user 1.69 s, sys: 39.9 ms, total: 1.73 s
Wall time: 1.74 s


In [72]:
print("Список санкций:")
sanctions.sort()
ssss = []
a = 0
for s in sanctions:
    #print(type(s))
    #a += 1;
    #print(s)
    n = Mystem().lemmatize(s)
    sankc = n[0]
    if not(sankc in ssss):
        a +=1
        print("" + str(a) + "." + sankc)
        ssss.append(sankc)    
    #ssss.append(sankc)
    #ssss.append(Mystem().lemmatize(s.split()[0]))
    #if a > 20:
    #    break

Список санкций:
1.американский
2.аналогичный
3.аяшский
4.балтийский
5.вильнюсский
6.возможный
7.высокий
8.газпром
9.геологический
10.дальний
11.долгинск
12.донецкий
13.европейский
14.единый
15.керченский
16.крупный
17.луганский
18.международный
19.морской
20.московский
21.национальный
22.независимый
23.новый
24.общий
25.один
26.пекин
27.первый
28.петербургский
29.печорский
30.последний
31.приразломный
32.прокладка
33.российский
34.северный
35.стокгольмский
36.таможенный
37.троицкий
38.турецкий
39.украинуть
40.центральный
41.челябинск
42.черный
43.чистый
44.эффективный
45.южнокиринский
46.ярославский
47.аграрный
48.административный
49.азиатский
50.англоголландский
51.антикризисный
52.антимонопольный
53.антироссийский
54.арктический
55.базовый
56.беспрецедентный
57.близкий
58.большой
59.бывший
60.бюрократический
61.верхний
62.визовый
63.виртуальный
64.внебюджетный
65.внешний
66.внутренний
67.волатильный
68.восточный
69.вредный
70.второй
71.высококонкурентный
72.газовый
73.газоконденсатны

#### Часть 2. Классификационная

Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия. Для этого:

    бинаризуйте признак "цена закрытия": новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
    составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем:

    ClosingPrice_bin – бинарый целевой признак
    слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части.

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [73]:
y = D['ClosingPrice_bin']

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
df = vectorizer.fit_transform(D['lem_text'])

In [80]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LogReg = LogisticRegression()
preds = LogReg.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.44      0.43      0.43       170
          1       0.54      0.55      0.54       208

avg / total       0.49      0.49      0.49       378

